In [1]:
import json
import os
import pandas as pd
import numpy as np
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

In [2]:
file_path = "../data/squad-train-v1.1.json"
with open(file_path, 'r') as f:
    data = json.load(f)

In [3]:
data['data'][0]['paragraphs'][0]['qas'][0]['answers']

[{'answer_start': 515, 'text': 'Saint Bernadette Soubirous'}]

In [4]:
questions = []
contexts = []
answers = []
answer_starts = []
for i in data['data']:
    for paragraph in i['paragraphs']:
        for qas in paragraph['qas']:
            questions.append(qas['question'])
            contexts.append(paragraph['context'])
            answers.append(qas['answers'][0]['text'])
            answer_starts.append(qas['answers'][0]['answer_start'])

In [5]:
df = pd.DataFrame({'question': questions, 'context': contexts, 'answer': answers, 'answer_start': answer_starts})
df.head()

,question,context,answer,answer_start
0,To whom did the Virgin Mary allegedly appear i...,"Architecturally, the school has a Catholic cha...",Saint Bernadette Soubirous,515
1,What is in front of the Notre Dame Main Building?,"Architecturally, the school has a Catholic cha...",a copper statue of Christ,188
2,The Basilica of the Sacred heart at Notre Dame...,"Architecturally, the school has a Catholic cha...",the Main Building,279
3,What is the Grotto at Notre Dame?,"Architecturally, the school has a Catholic cha...",a Marian place of prayer and reflection,381
4,What sits on top of the Main Building at Notre...,"Architecturally, the school has a Catholic cha...",a golden statue of the Virgin Mary,92


In [18]:
df.to_csv('../data/squad-train-v1.1.csv', index=False)

In [6]:
# test first five rows of data
test = df.iloc[:5]

In [28]:
inputs = tokenizer(test['question'].tolist(), test['context'].tolist(), truncation='only_second',
                   stride=50, return_overflowing_tokens=True, return_offsets_mapping=True)
print(f"The 5 examples gave {len(inputs['input_ids'])} features.")
print(f"Here is where each comes from: {inputs['overflow_to_sample_mapping']}.")

The 5 examples gave 5 features.
Here is where each comes from: [0, 1, 2, 3, 4].


In [44]:
start_positions = []
end_positions = []

for i, offset in enumerate(inputs["offset_mapping"]):
    sample_idx = inputs["overflow_to_sample_mapping"][i]
    answer = test['answer'][sample_idx]
    start_char = test["answer_start"][sample_idx]
    end_char = test["answer_start"][sample_idx] + len(answer)
    print(start_char, end_char)
    print(answer)
    sequence_ids = inputs.sequence_ids(i)

    # Find the start and end of the context]
    idx = 0
    while sequence_ids[idx] != 1:
        idx += 1
    context_start = idx
    while sequence_ids[idx] == 1:
        idx += 1
    context_end = idx - 1
    print(context_start, context_end)

    # If the answer is not fully inside the context, label is (0, 0)
    if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
        start_positions.append(0)
        end_positions.append(0)
    else:
        # Otherwise it's the start and end token positions
        adder = next(i for i in range(len(inputs['input_ids'][0])) if inputs['input_ids'][0][i] == 102)
        idx = context_start
        while idx <= context_end and offset[idx][0 ] <= start_char:
            idx += 1
        start_positions.append(adder + idx - 1)

        idx = context_end
        while idx >= context_start and offset[idx][1] >= end_char:
            idx -= 1
        end_positions.append(adder + idx + 1)

start_positions, end_positions

515 541
Saint Bernadette Soubirous
17 179
188 213
a copper statue of Christ
13 175
279 296
the Main Building
17 179
381 420
a Marian place of prayer and reflection
12 174
92 126
a golden statue of the Virgin Mary
14 176


([152, 69, 99, 117, 50], [158, 73, 101, 123, 56])

In [42]:
idx = 2
sample_idx = inputs["overflow_to_sample_mapping"][idx]
answer = test['answer'][idx]

start = start_positions[idx] 
end = end_positions[idx] 
print(start, end)
labeled_answer = tokenizer.decode(inputs["input_ids"][idx][start : end + 1])

print(f"Theoretical answer: {answer}, labels give: {labeled_answer}")

99 101
Theoretical answer: the Main Building, labels give: ##lica is the


In [19]:
def preprocess_function(test):
    inputs = tokenizer(
        test['question'].tolist(),
        test['context'].tolist(),
        max_length=384,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    answers = test['answer'].tolist()
    answer_starts = test['answer_start'].tolist()
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        start_char = answer_starts[i]
        end_char = start_char + len(answers[i])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [20]:
inp = preprocess_function(test)
starts = inp["start_positions"]
ends = inp["end_positions"]
for ind, (s, e) in enumerate(zip(starts, ends)):
    print(tokenizer.decode(inp["input_ids"][ind][s:e+1]))

Saint Bernadette Soubirous
a copper statue of Christ
the Main Building
a Marian place of prayer and reflection
a golden statue of the Virgin Mary


In [16]:
test.answer

0                 Saint Bernadette Soubirous
1                  a copper statue of Christ
2                          the Main Building
3    a Marian place of prayer and reflection
4         a golden statue of the Virgin Mary
Name: answer, dtype: object